This chapter covers
- The Keras functional API
- Using Keras callbacks
- Working with the TensorBoard visualization tool
- Important best practices for developing SOTA models

本章是关于Keras高级特性的介绍，使得你在应对deep-learning practice时更加得心应手。实践中包括了batch normalization, residual connections, hyperparameter optimization and model ensembling.

### 7.1 Going beyond the Sequential model: the Keras functional API

